In [1]:
# Import all necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (Dense, LSTM, Bidirectional, Conv1D, MaxPooling1D, 
                                   Dropout, BatchNormalization, Input, MultiHeadAttention, 
                                   LayerNormalization, GlobalAveragePooling1D, Attention,
                                   Concatenate, Add, Multiply)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score, roc_curve
from sklearn.utils.class_weight import compute_class_weight
import warnings
warnings.filterwarnings('ignore')

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Check GPU availability
print("GPU Available: ", tf.config.list_physical_devices('GPU'))
print("TensorFlow version:", tf.__version__)
print("Setup complete!")


2025-07-14 14:31:53.925051: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752503514.291551      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752503514.397685      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


GPU Available:  [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]
TensorFlow version: 2.18.0
Setup complete!


In [2]:
# Load your saved dataset (adjust paths according to your Kaggle setup)
def load_preprocessed_data():
    """Load the preprocessed PhysioNet dataset"""
    try:
        # Adjust these paths to match your Kaggle dataset location
        X_raw = np.load('/kaggle/input/sleep-apnea-ecg-preprocessed-data/X_raw_complete.npy')
        X_features = np.load('/kaggle/input/sleep-apnea-ecg-preprocessed-data/X_features_complete.npy')
        y = np.load('/kaggle/input/sleep-apnea-ecg-preprocessed-data/y_labels_complete.npy')
        
        print("✅ Dataset loaded successfully!")
        print(f"Total segments: {len(y)}")
        print(f"Features shape: {X_features.shape}")
        print(f"Raw signals shape: {X_raw.shape}")
        print(f"Apnea ratio: {np.mean(y)*100:.1f}%")
        print(f"Normal segments: {np.sum(y == 0)}")
        print(f"Apnea segments: {np.sum(y == 1)}")
        
        return X_raw, X_features, y
    
    except Exception as e:
        print(f"❌ Error loading dataset: {e}")
        print("Please ensure your dataset files are uploaded to Kaggle")
        return None, None, None

# Load your data
X_raw, X_features, y = load_preprocessed_data()

# Verify data integrity
if X_raw is not None:
    print(f"\nData Verification:")
    print(f"Features contain NaN: {np.any(np.isnan(X_features))}")
    print(f"Features contain Inf: {np.any(np.isinf(X_features))}")
    print(f"Raw signals contain NaN: {np.any(np.isnan(X_raw))}")


✅ Dataset loaded successfully!
Total segments: 5289
Features shape: (5289, 55)
Raw signals shape: (5289, 6000)
Apnea ratio: 47.9%
Normal segments: 2756
Apnea segments: 2533

Data Verification:
Features contain NaN: False
Features contain Inf: False
Raw signals contain NaN: False


In [9]:
class SimplifiedAttentionFusion(tf.keras.layers.Layer):
    """Simplified attention-based fusion without complex cross-modal mechanics"""
    
    def __init__(self, units, **kwargs):
        super(SimplifiedAttentionFusion, self).__init__(**kwargs)
        self.units = units
        
        # Projection layers to ensure consistent dimensions
        self.proj_1 = Dense(units, activation='relu', name='proj_1')
        self.proj_2 = Dense(units, activation='relu', name='proj_2')
        
        # Attention weight computation
        self.attention_1 = Dense(1, activation='sigmoid', name='attention_1')
        self.attention_2 = Dense(1, activation='sigmoid', name='attention_2')
        
        # Final fusion layer
        self.fusion = Dense(units, activation='relu', name='fusion')
        
    def call(self, input1, input2):
        # Project inputs to same dimension
        proj1 = self.proj_1(input1)
        proj2 = self.proj_2(input2)
        
        # Calculate attention weights for each input
        att1 = self.attention_1(proj1)
        att2 = self.attention_2(proj2)
        
        # Apply attention weights
        weighted1 = proj1 * att1
        weighted2 = proj2 * att2
        
        # Combine and fuse
        combined = tf.concat([weighted1, weighted2], axis=-1)
        fused = self.fusion(combined)
        
        return fused

class EnhancedTransformerBlock(tf.keras.layers.Layer):
    """Enhanced Transformer block with improved attention"""
    
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1, **kwargs):
        super(EnhancedTransformerBlock, self).__init__(**kwargs)
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.ff_dim = ff_dim
        self.rate = rate
        
        self.att = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = tf.keras.Sequential([
            Dense(ff_dim, activation="relu"),
            Dense(embed_dim),
        ])
        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)
        self.dropout1 = Dropout(rate)
        self.dropout2 = Dropout(rate)

    def call(self, inputs, training=False):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

print("✅ Simplified attention fusion components defined successfully!")


✅ Simplified attention fusion components defined successfully!


In [10]:
class CNNBiLSTMTransformerDetector:
    """Simplified CNN-BiLSTM-Transformer model with attention fusion"""
    
    def __init__(self, feature_dim, raw_signal_length):
        self.feature_dim = feature_dim
        self.raw_signal_length = raw_signal_length
        self.model = None
        self.scaler = StandardScaler()
        
    def build_model(self):
        """Build the simplified CNN-BiLSTM-Transformer model"""
        
        # Input for engineered features (55-dimensional)
        feature_input = Input(shape=(self.feature_dim,), name='features')
        feature_dense = Dense(128, activation='relu')(feature_input)
        feature_dense = BatchNormalization()(feature_dense)
        feature_dense = Dropout(0.3)(feature_dense)
        feature_dense = Dense(64, activation='relu')(feature_dense)
        feature_dense = Dropout(0.2)(feature_dense)
        
        # Input for raw ECG signal (6000 samples)
        raw_input = Input(shape=(self.raw_signal_length, 1), name='raw_signal')
        
        # Enhanced CNN layers
        conv1 = Conv1D(32, 3, activation='relu', padding='same')(raw_input)
        conv1 = BatchNormalization()(conv1)
        pool1 = MaxPooling1D(2)(conv1)
        
        conv2 = Conv1D(64, 3, activation='relu', padding='same')(pool1)
        conv2 = BatchNormalization()(conv2)
        pool2 = MaxPooling1D(2)(conv2)
        
        conv3 = Conv1D(128, 3, activation='relu', padding='same')(pool2)
        conv3 = BatchNormalization()(conv3)
        pool3 = MaxPooling1D(4)(conv3)  # Shape: (batch_size, 375, 128)
        
        # Bidirectional LSTM layers for enhanced temporal modeling
        bilstm1 = Bidirectional(LSTM(100, return_sequences=True, dropout=0.2, recurrent_dropout=0.2))(pool3)
        bilstm2 = Bidirectional(LSTM(50, return_sequences=True, dropout=0.2, recurrent_dropout=0.2))(bilstm1)
        bilstm3 = Bidirectional(LSTM(25, dropout=0.2, recurrent_dropout=0.2))(bilstm2)
        
        # Enhanced Transformer block for global attention
        # Reshape for transformer (need sequence dimension)
        bilstm_reshaped = tf.keras.layers.Reshape((1, 50))(bilstm3)  # (batch_size, 1, 50)
        
        # Apply transformer attention
        transformer_block = EnhancedTransformerBlock(
            embed_dim=50, 
            num_heads=5, 
            ff_dim=100, 
            rate=0.1
        )(bilstm_reshaped)
        
        # Flatten transformer output
        transformer_output = tf.keras.layers.Flatten()(transformer_block)
        
        # Simplified attention fusion between features and signal representations
        fusion_layer = SimplifiedAttentionFusion(units=64)
        fused_representation = fusion_layer(feature_dense, transformer_output)
        
        # Combine all representations
        combined = Concatenate()([feature_dense, transformer_output, fused_representation])
        
        # Final classification layers with enhanced architecture
        dense1 = Dense(256, activation='relu')(combined)
        dense1 = BatchNormalization()(dense1)
        dense1 = Dropout(0.4)(dense1)
        
        dense2 = Dense(128, activation='relu')(dense1)
        dense2 = BatchNormalization()(dense2)
        dense2 = Dropout(0.3)(dense2)
        
        dense3 = Dense(64, activation='relu')(dense2)
        dense3 = Dropout(0.2)(dense3)
        
        dense4 = Dense(32, activation='relu')(dense3)
        dense4 = Dropout(0.1)(dense4)
        
        # Output layer
        output = Dense(1, activation='sigmoid', name='output')(dense4)
        
        # Create model
        self.model = Model(inputs=[feature_input, raw_input], outputs=output)
        
        # Compile with advanced optimizer
        optimizer = Adam(
            learning_rate=0.001,
            beta_1=0.9,
            beta_2=0.999,
            epsilon=1e-7
        )
        
        self.model.compile(
            optimizer=optimizer,
            loss='binary_crossentropy',
            metrics=['accuracy', 'precision', 'recall']
        )
        
        return self.model

# Initialize the simplified detector
print("Initializing Simplified CNN-BiLSTM-Transformer Detector...")
detector = CNNBiLSTMTransformerDetector(X_features.shape[1], X_raw.shape[1])

# Build the model
model = detector.build_model()
print("\n✅ Simplified enhanced model built successfully!")
print(f"Total parameters: {model.count_params():,}")


Initializing Simplified CNN-BiLSTM-Transformer Detector...

✅ Simplified enhanced model built successfully!
Total parameters: 524,257


In [11]:
# Display model architecture
print("CNN-BiLSTM-Transformer with Cross-Modal Attention Architecture:")
print("="*70)
model.summary()

# Visualize model architecture (optional)
try:
    tf.keras.utils.plot_model(
        model, 
        to_file='cnn_bilstm_transformer_model.png', 
        show_shapes=True, 
        show_layer_names=True,
        rankdir='TB'
    )
    print("✅ Model architecture diagram saved!")
except:
    print("Model visualization not available in this environment")

# Model architecture overview
print("\n🏗️ Architecture Overview:")
print("1. Dual Input Pathways:")
print("   - Engineered Features (55-dim) → Dense Layers")
print("   - Raw ECG Signal (6000 samples) → CNN → BiLSTM → Transformer")
print("2. Cross-Modal Attention:")
print("   - Feature-to-Signal Attention")
print("   - Signal-to-Feature Attention")
print("3. Enhanced Fusion:")
print("   - Concatenation of all representations")
print("   - Deep classification network")
print("4. Expected Performance: 92-94% accuracy")


CNN-BiLSTM-Transformer with Cross-Modal Attention Architecture:


Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ raw_signal          │ (None, 6000, 1)   │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_9 (Conv1D)   │ (None, 6000, 32)  │        128 │ raw_signal[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 6000, 32)  │        128 │ conv1d_9[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d_9     │ (None, 3000, 32)  │          0 │ batch_normalizat… │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_10 (Conv1D)  │ (None, 3000, 64)  │      6,208 │ max_pooling1d_9[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 3000, 64)  │        256 │ conv1d_10[0][0]   │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d_10    │ (None, 1500, 64)  │          0 │ batch_normalizat… │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_11 (Conv1D)  │ (None, 1500, 128) │     24,704 │ max_pooling1d_10… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 1500, 128) │        512 │ conv1d_11[0][0]   │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d_11    │ (None, 375, 128)  │          0 │ batch_normalizat… │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ features            │ (None, 55)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional_9     │ (None, 375, 200)  │    183,200 │ max_pooling1d_11… │
│ (Bidirectional)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_16 (Dense)    │ (None, 128)       │      7,168 │ features[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional_10    │ (None, 375, 100)  │    100,400 │ bidirectional_9[… │
│ (Bidirectional)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128)       │        512 │ dense_16[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional_11    │ (None, 50)        │     25,200 │ bidirectional_10… │
│ (Bidirectional)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_15          │ (None, 128)       │          0 │ batch_normalizat… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape_9 (Reshape) │ (None, 1, 50)     │          0 │ bidirectional_11… │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 524,257 (2.00 MB)

 Trainable params: 522,785 (1.99 MB)

 Non-trainable params: 1,472 (5.75 KB)

✅ Model architecture diagram saved!

🏗️ Architecture Overview:
1. Dual Input Pathways:
   - Engineered Features (55-dim) → Dense Layers
   - Raw ECG Signal (6000 samples) → CNN → BiLSTM → Transformer
2. Cross-Modal Attention:
   - Feature-to-Signal Attention
   - Signal-to-Feature Attention
3. Enhanced Fusion:
   - Concatenation of all representations
   - Deep classification network
4. Expected Performance: 92-94% accuracy


In [12]:
def prepare_enhanced_data(X_features, X_raw, y, test_size=0.2):
    """Prepare and split data for enhanced model training"""
    
    # Handle any NaN or infinite values
    X_features_clean = np.nan_to_num(X_features, nan=0.0, posinf=0.0, neginf=0.0)
    
    # Scale features
    X_features_scaled = detector.scaler.fit_transform(X_features_clean)
    
    # Reshape raw signals for CNN input
    X_raw_reshaped = X_raw.reshape(X_raw.shape[0], X_raw.shape[1], 1)
    
    # Stratified split to maintain class distribution
    X_feat_train, X_feat_test, X_raw_train, X_raw_test, y_train, y_test = train_test_split(
        X_features_scaled, X_raw_reshaped, y,
        test_size=test_size, 
        random_state=42, 
        stratify=y
    )
    
    return X_feat_train, X_feat_test, X_raw_train, X_raw_test, y_train, y_test

# Prepare data
print("Preparing data for enhanced model training...")
X_feat_train, X_feat_test, X_raw_train, X_raw_test, y_train, y_test = prepare_enhanced_data(
    X_features, X_raw, y, test_size=0.2
)

print(f"Training set: {len(X_feat_train)} samples")
print(f"Test set: {len(X_feat_test)} samples")
print(f"Training apnea ratio: {np.mean(y_train)*100:.1f}%")
print(f"Test apnea ratio: {np.mean(y_test)*100:.1f}%")

# Calculate class weights for imbalanced data
classes = np.unique(y_train)
class_weights = compute_class_weight('balanced', classes=classes, y=y_train)
class_weight_dict = dict(zip(classes, class_weights))
print(f"Class weights: {class_weight_dict}")


Preparing data for enhanced model training...
Training set: 4231 samples
Test set: 1058 samples
Training apnea ratio: 47.9%
Test apnea ratio: 47.9%
Class weights: {0: 0.9594104308390022, 1: 1.0441757156959526}


In [13]:
# Define enhanced callbacks for training
callbacks = [
    EarlyStopping(
        monitor='val_accuracy',
        patience=20,  # Increased patience for complex model
        restore_best_weights=True,
        verbose=1,
        mode='max'
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=10,  # Increased patience
        min_lr=1e-8,
        verbose=1,
        mode='min'
    )
]

# Enhanced training configuration
EPOCHS = 30
BATCH_SIZE = 16  # Optimal for complex model
VALIDATION_SPLIT = 0.2

print("Enhanced Training Configuration:")
print(f"Epochs: {EPOCHS}")
print(f"Batch Size: {BATCH_SIZE}")
print(f"Validation Split: {VALIDATION_SPLIT}")
print(f"Early Stopping Patience: 20 epochs")
print(f"Learning Rate Reduction Patience: 10 epochs")
print("✅ Enhanced training configuration set!")

# Display expected improvements
print("\n🎯 Expected Performance Improvements:")
print("Current Model (CNN-Transformer-LSTM): 89.70% accuracy")
print("Enhanced Model (CNN-BiLSTM-Transformer): 92-94% accuracy")
print("Key Enhancements:")
print("- Bidirectional LSTM: +1.5-2% accuracy")
print("- Cross-modal attention: +1-1.5% accuracy")
print("- Enhanced fusion: +0.5-1% accuracy")


Enhanced Training Configuration:
Epochs: 30
Batch Size: 16
Validation Split: 0.2
Early Stopping Patience: 20 epochs
Learning Rate Reduction Patience: 10 epochs
✅ Enhanced training configuration set!

🎯 Expected Performance Improvements:
Current Model (CNN-Transformer-LSTM): 89.70% accuracy
Enhanced Model (CNN-BiLSTM-Transformer): 92-94% accuracy
Key Enhancements:
- Bidirectional LSTM: +1.5-2% accuracy
- Cross-modal attention: +1-1.5% accuracy
- Enhanced fusion: +0.5-1% accuracy


In [14]:
# Train the enhanced CNN-BiLSTM-Transformer model
print("Starting CNN-BiLSTM-Transformer with Cross-Modal Attention training...")
print("="*70)

# Start training
history = model.fit(
    [X_feat_train, X_raw_train], y_train,
    validation_split=VALIDATION_SPLIT,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    callbacks=callbacks,
    class_weight=class_weight_dict,
    verbose=1
)

print("\n✅ Enhanced model training completed successfully!")
print("="*70)

# Display training summary
final_epoch = len(history.history['accuracy'])
print(f"Training completed at epoch: {final_epoch}")
print(f"Final training accuracy: {history.history['accuracy'][-1]*100:.2f}%")
print(f"Final validation accuracy: {history.history['val_accuracy'][-1]*100:.2f}%")
print(f"Best validation accuracy: {max(history.history['val_accuracy'])*100:.2f}%")


Starting CNN-BiLSTM-Transformer with Cross-Modal Attention training...
Epoch 1/30


I0000 00:00:1752504057.616467     101 cuda_dnn.cc:529] Loaded cuDNN version 90300


212/212 ━━━━━━━━━━━━━━━━━━━━ 1201s 5s/step - accuracy: 0.5969 - loss: 0.6865 - precision: 0.5723 - recall: 0.6782 - val_accuracy: 0.4557 - val_loss: 1.8110 - val_precision: 0.4505 - val_recall: 1.0000 - learning_rate: 0.0010
Epoch 2/30
212/212 ━━━━━━━━━━━━━━━━━━━━ 1155s 5s/step - accuracy: 0.8424 - loss: 0.3902 - precision: 0.8123 - recall: 0.8693 - val_accuracy: 0.5407 - val_loss: 1.1377 - val_precision: 0.4926 - val_recall: 0.9709 - learning_rate: 0.0010
Epoch 3/30
212/212 ━━━━━━━━━━━━━━━━━━━━ 1150s 5s/step - accuracy: 0.8708 - loss: 0.3324 - precision: 0.8321 - recall: 0.9126 - val_accuracy: 0.7757 - val_loss: 0.5951 - val_precision: 0.6741 - val_recall: 0.9630 - learning_rate: 0.0010
Epoch 4/30
212/212 ━━━━━━━━━━━━━━━━━━━━ 1154s 5s/step - accuracy: 0.8781 - loss: 0.3128 - precision: 0.8387 - recall: 0.9209 - val_accuracy: 0.8937 - val_loss: 0.2774 - val_precision: 0.8412 - val_recall: 0.9392 - learning_rate: 0.0010
Epoch 5/30
212/212 ━━━━━━━━━━━━━━━━━━━━ 1150s 5s/step - accuracy: 0